# Session 10: Prompt Logic and Human-AI Interaction
## Controlling Models Through Design

**Session Length:** 2 hours

**Today's Mission:** Master prompt engineering -- the skill of designing inputs to get better outputs from AI models. Then use everything you have learned to plan your independent project.

### Session Outline
| Time | Activity |
|------|----------|
| 0:00-0:05 | Review: What pipeline designs did you sketch? |
| 0:05-0:30 | Part 1: Prompts as Model Control |
| 0:30-1:00 | Part 2: Designing a Tool (live build with AI) |
| 1:00-1:40 | Part 3: Project Planning |
| 1:40-2:00 | On Your Own: Start building your project prototype |

### Key Vocabulary
| Term | Definition |
|------|-----------|
| Prompt Engineering | Designing inputs to get better outputs from models |
| CLEAR Framework | Context, Language, Explain, Ask, Requirements |
| Zero-Shot Prompting | Giving a model a task without examples |
| Candidate Labels | The categories you give a zero-shot model to choose from |
| Tool Design | Planning how an AI tool should work for its users |

---

## Review: What Pipeline Designs Did You Sketch? (0:00-0:05)

Last session we built multi-model pipelines -- chaining models so the output of one feeds into the next. We saw how error cascades can ruin downstream results, and we designed pipelines on paper.

Today we focus on the **control layer** -- how the inputs you design determine the outputs you get. In Session 5, you learned about temperature and top_p. Those are numerical controls. But the most powerful control is the **prompt itself**.

Now that you understand training data (Session 4), bias (Session 8), and generalization (Session 7), you can write better prompts. Let's see how.

---

## Setup

Run this cell to install the libraries we need.

In [ ]:
!pip install transformers==4.47.1 -q

### Important: Restart Your Runtime

After installing packages, you need to restart the runtime so Python can find them.

**Go to: Runtime > Restart runtime**

After restarting, come back here and continue running the cells below. You do NOT need to re-run the install cell -- the packages are already installed. Just start from the next code cell.

---

In [ ]:
from transformers import pipeline
print("Ready!")

---

## Part 1: Prompts as Model Control (0:05-0:30)

The zero-shot classifier does not look at code. It looks at **the labels you give it**. Change the labels, change the results. This makes the labels a form of **prompt engineering** -- you are controlling the model's behavior through your input design.

### Experiment 1: Vague vs. Specific Categories

In [ ]:
classifier = pipeline("zero-shot-classification")
print("Classifier loaded!")

In [ ]:
text = "I spent three hours on this assignment and I'm exhausted but proud of the result"

# Vague categories
vague_labels = ["good", "bad"]
result_vague = classifier(text, vague_labels)

# Specific categories
specific_labels = ["productive effort", "frustration", "accomplishment", "burnout"]
result_specific = classifier(text, specific_labels)

print("VAGUE vs. SPECIFIC CATEGORIES")
print("=" * 55)
print(f"Text: {text}\n")

print("Vague labels:")
for label, score in zip(result_vague['labels'], result_vague['scores']):
    print(f"  {label}: {score:.1%}")

print("\nSpecific labels:")
for label, score in zip(result_specific['labels'], result_specific['scores']):
    print(f"  {label}: {score:.1%}")

print("\nSame text, completely different analysis -- just by changing the labels.")

The vague labels force a binary choice that misses the complexity. The specific labels capture the nuance -- this text contains BOTH exhaustion AND pride. **Better labels produce better insights.**

### Experiment 2: One-Word vs. Descriptive Labels

In [ ]:
text = "The new policy will increase costs for small businesses but could create more jobs overall"

# One-word labels
one_word = ["positive", "negative", "neutral"]
result_short = classifier(text, one_word)

# Descriptive labels
descriptive = [
    "mostly beneficial with some drawbacks",
    "mostly harmful with some benefits",
    "mixed impact with trade-offs",
    "insufficient information to judge"
]
result_desc = classifier(text, descriptive)

print("ONE-WORD vs. DESCRIPTIVE LABELS")
print("=" * 55)
print(f"Text: {text}\n")

print("One-word labels:")
for label, score in zip(result_short['labels'], result_short['scores']):
    print(f"  {label}: {score:.1%}")

print("\nDescriptive labels:")
for label, score in zip(result_desc['labels'], result_desc['scores']):
    print(f"  {label}: {score:.1%}")

Descriptive labels give the model more context about what each category actually means. This is prompt engineering at its core: **the more precisely you describe what you want, the more precisely the model can deliver.**

### Experiment 3: Student-Designed Labels

> **INSTRUCTOR NOTE:** Ask students to suggest a text and then design their own set of labels. Run it and discuss whether the labels captured what they wanted.

In [ ]:
student_text = "REPLACE WITH STUDENT SUGGESTION"

student_categories = ["REPLACE", "WITH", "YOUR", "OWN"]

if "REPLACE" not in student_text and "REPLACE" not in student_categories[0]:
    result = classifier(student_text, student_categories)
    print(f"Text: {student_text}\n")
    print("Your categories:")
    for label, score in zip(result['labels'], result['scores']):
        print(f"  {label}: {score:.1%}")

### Experiment 4: Prompt Phrasing for Text Generation

In [ ]:
generator = pipeline("text-generation", model="distilgpt2")
print("Generator loaded!")

In [ ]:
# Same topic, different prompt framings
prompts = [
    "Artificial intelligence is",
    "The most important thing about artificial intelligence is",
    "A scientist explaining artificial intelligence to a child would say",
    "The danger of artificial intelligence is",
    "The promise of artificial intelligence is",
]

print("PROMPT FRAMING EXPERIMENT")
print("=" * 60)

for prompt in prompts:
    result = generator(prompt, max_length=50, do_sample=True, temperature=0.8)
    output = result[0]['generated_text'][len(prompt):].strip()[:60]
    print(f"\nPrompt: \"{prompt}\"")
    print(f"  --> {output}")

Same topic, five completely different directions -- all controlled by how you frame the prompt. The prompt is not just input. It is **steering**.

### The Key Insight

| What You Learned | Session | How It Connects to Prompts |
|-----------------|---------|---------------------------|
| Models have training domains | Session 7 | Use language that matches the training domain |
| Models have biases | Session 8 | Test prompts for biased framing |
| Labels change classification | Session 5 | Design labels carefully |
| Error cascades | Session 9 | Bad prompts cascade into bad outputs |

Prompt engineering is not a trick. It is **applied understanding of how models work**.

---

> **ASK AI ABOUT THIS**
>
> Copy the vague vs. specific experiment into Claude or ChatGPT and ask:
>
> *"Why do more specific category labels produce better results in zero-shot classification? What is the model actually doing with these labels internally?"*
>
> This is how real programmers learn -- by asking questions about code they encounter.

---

## Part 2: Designing a Tool (0:30-1:00)

You know what models can do. You know their limits. You know how to chain them. You know how to control them with prompts. Now: **what do you want to BUILD?**

### The CLEAR Framework (Revisited)

In Session 3, you learned the CLEAR framework for asking AI to write code:

| Letter | Meaning | Example |
|--------|---------|---------|
| **C** | Context | "I'm working in Google Colab with Python" |
| **L** | Language/Libraries | "Using the Hugging Face transformers library" |
| **E** | Explain the goal | "I want to build a tool that analyzes customer reviews" |
| **A** | Ask specifically | "Write a function that takes a list of reviews and returns sentiment + emotion for each" |
| **R** | Requirements | "Include comments, handle empty input, print a summary at the end" |

Back then, you were learning the basics. Now you have 9 sessions of knowledge about what models can do, how they fail, and how to control them. Your CLEAR prompts can be **much more sophisticated**.

### Live Demo: Building a Tool With AI

> **INSTRUCTOR NOTE:** This is the core exercise. Open Claude or ChatGPT on screen. Ask a student to describe a tool they want to build. Write a CLEAR prompt WITH the class -- have them suggest each piece. Paste the code into Colab. Run it. Debug if needed. The entire Part 2 IS this demo.
>
> **Example CLEAR prompt the class might write together:**
>
> "I'm working in Google Colab with Python. Using the Hugging Face transformers library, write a function called `analyze_feedback` that:
> - Takes a list of student feedback comments as input
> - Uses sentiment analysis to classify each as positive/negative
> - Uses zero-shot classification with labels ["praise", "suggestion", "complaint", "question"]
> - Prints each comment with its sentiment and category
> - Prints a summary at the end (counts per category, overall sentiment distribution)
> - Include comments explaining each step"
>
> Type the AI-generated code in the cell below and run it live.

In [ ]:
# PASTE AI-GENERATED CODE HERE
# (Instructor: paste the code that Claude/ChatGPT generates from the class CLEAR prompt)

# Placeholder example -- replace with the live-generated code:
def analyze_feedback(comments):
    """Analyze student feedback using sentiment + zero-shot classification."""
    sent = pipeline("sentiment-analysis")
    clf = pipeline("zero-shot-classification")
    categories = ["praise", "suggestion", "complaint", "question"]

    print("FEEDBACK ANALYSIS")
    print("=" * 55)

    cat_counts = {c: 0 for c in categories}
    sent_counts = {"POSITIVE": 0, "NEGATIVE": 0}

    for comment in comments:
        s = sent(comment)[0]
        c = clf(comment, categories)
        top_cat = c['labels'][0]
        cat_counts[top_cat] += 1
        sent_counts[s['label']] += 1

        print(f"\n\"{comment}\"")
        print(f"  Sentiment: {s['label']} ({s['score']:.0%})")
        print(f"  Category: {top_cat} ({c['scores'][0]:.0%})")

    print("\n" + "=" * 55)
    print("SUMMARY")
    print(f"  Total comments: {len(comments)}")
    for cat, count in cat_counts.items():
        if count > 0:
            print(f"  {cat}: {count}")
    print(f"  Positive: {sent_counts['POSITIVE']}, Negative: {sent_counts['NEGATIVE']}")

# Test it
sample_feedback = [
    "The examples in class were really helpful!",
    "Could we get more time for the group project?",
    "I didn't understand the homework instructions at all.",
    "Is the final exam cumulative or just recent material?",
    "Best class I've taken this year, seriously.",
]

analyze_feedback(sample_feedback)

> **INSTRUCTOR NOTE:** After running the tool, discuss with the class:
> - What worked well?
> - What would you change?
> - How could we improve the CLEAR prompt to get better code?
> - What would happen if we changed the categories?

---

## Part 3: Project Planning (1:00-1:25)

By the end of this section, each student should have a **1-sentence project description** and a plan for what to build.

### What You Know Now

Over 10 sessions, you have learned:

| Session | Skill |
|---------|-------|
| 1 | INPUT --> MODEL --> OUTPUT pattern |
| 2 | Running models with code (pipelines) |
| 3 | Using AI to help you code (CLEAR framework) |
| 4 | How models learn (supervised learning, training data) |
| 5 | Controlling models (temperature, top-p, labels) |
| 6 | Evaluating models (confidence, accuracy, false positives) |
| 7 | Domain shift and generalization |
| 8 | Bias, uncertainty, and real-world consequences |
| 9 | Multi-model pipelines and error cascades |
| 10 | Prompt engineering and tool design |

You are ready to build something real.

### Project Spec Template

Fill this out. Be specific.

**My Project Name:**

**One-sentence description:** (What does it do, and who is it for?)

**What models will it use?**
- Model 1: _________ (for _________)
- Model 2: _________ (for _________)
- Model 3: _________ (for _________) [optional]

**What is the input?**

**What is the output?**

**One thing it probably WON'T handle well:** (Connect to Sessions 7-8 -- what domain shift or bias issues might affect your tool?)

**My CLEAR prompt to start building:**
```
C - Context:
L - Language/Libraries:
E - Explain the goal:
A - Ask specifically:
R - Requirements:
```

> **INSTRUCTOR NOTE:** Have each student share their 1-sentence project description with the group. This is a commitment point -- by next session, they should have something that runs.

### Browse Models for Your Project

> **INSTRUCTOR NOTE:** Have students go to huggingface.co/models and search for models relevant to their project. Show them how to filter by task (text-classification, summarization, etc.) and sort by downloads. Point out that popular models are popular for a reason -- they tend to be more reliable.

### Available Models Reference

Here are the models you have used in this course (all work on free Colab CPU):

**Text Models:**
| Pipeline | Model | What It Does |
|----------|-------|-------------|
| `sentiment-analysis` | (default) | Positive/negative sentiment |
| `sentiment-analysis` | `cardiffnlp/twitter-roberta-base-sentiment-latest` | Twitter-optimized sentiment |
| `sentiment-analysis` | `nlptown/bert-base-multilingual-uncased-sentiment` | 1-5 star ratings |
| `text-classification` | `j-hartmann/emotion-english-distilroberta-base` | Emotion detection (joy, anger, etc.) |
| `zero-shot-classification` | (default) | Classify into ANY categories you define |
| `question-answering` | (default) | Extract answers from a passage |
| `summarization` | `sshleifer/distilbart-cnn-12-6` | Summarize long text |
| `ner` | (default, `grouped_entities=True`) | Find people, places, organizations |
| `text-generation` | `distilgpt2` | Generate text from a prompt |

**Image Models:**
| Pipeline | Model | What It Does |
|----------|-------|-------------|
| `image-to-text` | `Salesforce/blip-image-captioning-base` | Describe an image in words |
| `zero-shot-image-classification` | `openai/clip-vit-base-patch32` | Classify images into ANY categories |

---

> **ASK AI ABOUT THIS**
>
> Describe your project idea to Claude or ChatGPT and ask:
>
> *"I want to build [your idea] using Hugging Face models in Google Colab. What models would you recommend? What order should they run in? What problems should I watch out for?"*
>
> This is how real programmers learn -- by asking questions about code they encounter.

---

## On Your Own (1:40-2:00)

### Start Building Your Project Prototype

Use the CLEAR framework to ask Claude or ChatGPT for starter code. Paste it here and try to get something running before next session.

In [ ]:
# YOUR PROJECT: STARTER CODE
# Paste AI-generated code here and test it

# Step 1: Load your models
# from transformers import pipeline
# model_1 = pipeline("...")
# model_2 = pipeline("...")

# Step 2: Your main function
def my_project(input_data):
    """
    REPLACE: Describe what your project does.
    """
    # YOUR CODE HERE
    pass

# Step 3: Test it
# my_project("test input here")

In [ ]:
# PROJECT TESTING CELL
# Try different inputs to see how your tool behaves

### Between Sessions

Come to Session 11 with **something that runs** -- even if it is rough. It does not need to be perfect. It needs to work.

Use Claude or ChatGPT with the CLEAR framework to help you build. That is not cheating -- it is exactly the skill we have been practicing.

---

### Checklist: Before You Leave

- [ ] Saw how label design changes zero-shot classification results
- [ ] Understood that prompt framing steers text generation
- [ ] Watched (or participated in) a live tool-building exercise with AI
- [ ] Wrote a 1-sentence project description
- [ ] Filled out the project spec template
- [ ] Browsed Hugging Face models for your project
- [ ] Started (or have a plan for) your project prototype
- [ ] Saved your work (File > Save a copy in Drive)

---

## Looking Ahead

Next session is **project work time**. You will share what you built, get feedback, iterate, and document. Come with something that runs -- even if it is messy. Session 11 is about making it better, not starting from scratch.

See you next session.

---

*Youth Horizons AI Researcher Program - Level 2*